In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import undetected_chromedriver
import pandas as pd
from time import sleep
from multiprocessing import Pool
from functools import partial
import os

In [2]:
def fetchMatch(matchID, driver):
    driver.get("https://csgostats.gg/match/" + str(matchID))
    
    mm = driver.find_element(By.XPATH,"//body/div[2]/div[1]/div[2]/div/div[1]/div[2]/div[1]/div")
    if(mm.get_attribute("textContent").strip()[:20] == "Official Matchmaking"):
        players =  driver.find_elements( By.CLASS_NAME, 'player-link')
        match = []
        for player in players:

            playerstats = {}
            playerstats["Match ID"] = matchID
            playerstats["Player ID"] = player.get_attribute('href')[-17:-1]

            PCard = player.find_element(By.XPATH,"./..").find_element(By.XPATH,"./..")

            rank = PCard.find_elements(By.CLASS_NAME, "rank")
            if rank.__len__():
                playerstats["Rank"] = (rank[0].get_attribute("src"))[-6:-4].strip("/")

            rankupicon = PCard.find_elements(By.XPATH, "./td/div/div/span")

            if(rankupicon.__len__()):
                rankup = rankupicon[0].get_attribute("class")
                if rankup == "glyphicon glyphicon-chevron-up" :
                    playerstats["Rank Gain"] = 1

                elif rankup == "glyphicon glyphicon-chevron-down" :
                    playerstats["Rank Gain"] = -1

            else :
                playerstats["Rank Gain"] = 0        
            
            # print(stats["rank gain"])
            teamScore = player.find_element(By.XPATH,"./..")
            teamScore = teamScore.find_element(By.XPATH,"./..")
            teamScore = teamScore.find_element(By.XPATH,"./..")
            
            #teamScore = teamScore.find_element(By.ID,'match-scoreboard')
            #teamScore = teamScore.find_element(By.CLASS_NAME,'scoreboard')
            
            playerstats["Team"] = teamScore.find_element(By.XPATH, "./tr/td/div").get_attribute("textContent").strip() [-1:]

            stats = PCard.find_elements(By.TAG_NAME, "td")

            # for stat in stats:
            #     print(stat.get_attribute("data-collapse"),stat.get_attribute("textContent").strip() )
            
            

            playerstats["K"] =                  stats[2].get_attribute("textContent").strip()
            playerstats["D"] =                  stats[3].get_attribute("textContent").strip()  
            playerstats["A"] =                  stats[4].get_attribute("textContent").strip()  
            playerstats["+/-"] =                stats[5].get_attribute("textContent").strip()  
            playerstats["K/D"] =                stats[6].get_attribute("textContent").strip()  
            playerstats["ADR"] =                stats[7].get_attribute("textContent").strip()  
            playerstats["HS%"] =                stats[8].get_attribute("textContent").strip()  
            playerstats["KAST"] =               stats[9].get_attribute("textContent").strip()  
            playerstats["Rating"] =             stats[10].get_attribute("textContent").strip()  
            playerstats["EF"] =                 stats[11].get_attribute("textContent").strip()  
            playerstats["FA"] =                 stats[12].get_attribute("textContent").strip()  
            playerstats["EBT"] =                stats[13].get_attribute("textContent").strip()  
            playerstats["UD"] =                 stats[14].get_attribute("textContent").strip()  
            playerstats["FKD"] =                stats[15].get_attribute("textContent").strip()  
            playerstats["FK"] =                 stats[16].get_attribute("textContent").strip() 
            playerstats["FD"] =                 stats[17].get_attribute("textContent").strip() 
            playerstats["FK as T"] =            stats[18].get_attribute("textContent").strip()  
            playerstats["FD as T"] =            stats[19].get_attribute("textContent").strip()  
            playerstats["FK as CT"] =           stats[20].get_attribute("textContent").strip()  
            playerstats["FD as CT"] =           stats[21].get_attribute("textContent").strip()  
            playerstats["Trade K"] =            stats[22].get_attribute("textContent").strip()  
            playerstats["Trade D"] =            stats[23].get_attribute("textContent").strip()  
            playerstats["Trade FK"] =           stats[24].get_attribute("textContent").strip()  
            playerstats["Trade FD"] =           stats[25].get_attribute("textContent").strip()  
            playerstats["Trade FK as T"] =      stats[26].get_attribute("textContent").strip()  
            playerstats["Trade FD as T"] =      stats[27].get_attribute("textContent").strip()  
            playerstats["Trade FK as CT"] =     stats[28].get_attribute("textContent").strip()  
            playerstats["Trade FD as CT"] =     stats[29].get_attribute("textContent").strip()  
            playerstats["1vX"] =                stats[30].get_attribute("textContent").strip()  
            playerstats["1v5"] =                stats[31].get_attribute("textContent").strip()  
            playerstats["1v4"] =                stats[32].get_attribute("textContent").strip()  
            playerstats["1v3"] =                stats[33].get_attribute("textContent").strip()  
            playerstats["1v2"] =                stats[34].get_attribute("textContent").strip()  
            playerstats["1v1"] =                stats[35].get_attribute("textContent").strip()  
            playerstats["3K+"] =                stats[36].get_attribute("textContent").strip()  
            playerstats["5K"] =                 stats[37].get_attribute("textContent").strip()  
            playerstats["4K"] =                 stats[38].get_attribute("textContent").strip()  
            playerstats["3K"] =                 stats[39].get_attribute("textContent").strip()  
            playerstats["2K"] =                 stats[40].get_attribute("textContent").strip()  
            playerstats["1K"] =                 stats[41].get_attribute("textContent").strip()  
        
            # print(stats["team"])
            match.append(playerstats)
        return match
        
    else :
        return []


In [3]:
def multiFetch(offset):
    
    columns = ['Match ID', 'Player ID', 'Rank', 'Rank Gain', 'Team', 'K', 'D', 'A',
        '+/-', 'K/D', 'ADR', 'HS%', 'KAST', 'Rating', 'EF', 'FA', 'EBT', 'UD',
        'FKD', 'FK', 'FD', 'FK as T', 'FD as T', 'FK as CT', 'FD as CT',
        'Trade K', 'Trade D', 'Trade FK', 'Trade FD', 'Trade FK as T',
        'Trade FD as T', 'Trade FK as CT', 'Trade FD as CT', '1vX', '1v5',
        '1v4', '1v3', '1v2', '1v1', '3K+', '5K', '4K', '3K', '2K', '1K']
    df = pd.DataFrame(columns= columns)

    # frame = []

    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    driver = undetected_chromedriver.Chrome(options= options)

    # driver.implicitly_wait(0.1)
    # driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

    for i in range(10):
        res = fetchMatch((offset + i),driver)
        # sleep(1)
        # print(res)
        df = pd.concat([pd.DataFrame(res,columns= columns),df])
        # driver.execute_script("window.open();")
        # driver.close()
        # driver.switch_to.window(driver.window_handles[-1])
        

    driver.quit()
    # print(frame)
    # df = pd.concat(frame)
    # display(df)
    return df

In [4]:
def testing(offset):
    print(offset)

In [5]:
columns = ['Match ID', 'Player ID', 'Rank', 'Rank Gain', 'Team', 'K', 'D', 'A',
        '+/-', 'K/D', 'ADR', 'HS%', 'KAST', 'Rating', 'EF', 'FA', 'EBT', 'UD',
        'FKD', 'FK', 'FD', 'FK as T', 'FD as T', 'FK as CT', 'FD as CT',
        'Trade K', 'Trade D', 'Trade FK', 'Trade FD', 'Trade FK as T',
        'Trade FD as T', 'Trade FK as CT', 'Trade FD as CT', '1vX', '1v5',
        '1v4', '1v3', '1v2', '1v1', '3K+', '5K', '4K', '3K', '2K', '1K']
        
df = pd.DataFrame(columns= columns)

nb = 10

nbarray = []

for i in range(0,nb * os.cpu_count()-1,nb):
    nbarray.append(i + 79000000)

print(nbarray)

if __name__ == '__main__':
    with Pool(os.cpu_count()-1) as p:
        p.map(testing,nbarray)

[79000000, 79000010, 79000020, 79000030, 79000040, 79000050, 79000060, 79000070, 79000080, 79000090, 79000100, 79000110, 79000120, 79000130, 79000140, 79000150]


In [ ]:

display(df)
df.to_csv("test.csv")

,Match ID,Player ID,Rank,Rank Gain,Team,K,D,A,+/-,K/D,...,1v4,1v3,1v2,1v1,3K+,5K,4K,3K,2K,1K
0,79288989,7656119919752391,10,0,1,12,10,2,2,1.20,...,0,0,0,0,2,0,0,2,1,4
1,79288989,7656119801257308,9,0,1,9,11,2,-2,0.80,...,0,0,0,0,1,0,0,1,2,2
2,79288989,7656119898039161,9,0,1,8,12,4,-4,0.70,...,0,0,0,0,1,0,1,0,1,2
3,79288989,7656119824090284,9,0,1,8,10,1,-2,0.80,...,0,0,0,0,1,0,0,1,1,3
4,79288989,7656119922736115,9,0,1,7,10,1,-3,0.70,...,0,0,0,0,1,0,0,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,79288998,7656119896248147,11,0,2,26,22,2,4,1.20,...,0,0,0,0,2,0,0,2,4,12
6,79288998,7656119911226308,12,0,2,26,20,4,6,1.30,...,0,0,0,1,1,0,0,1,3,17
7,79288998,7656119813839105,13,0,2,21,21,5,0,1.00,...,0,0,0,0,2,0,0,2,3,9
8,79288998,7656119882370902,12,0,2,19,22,6,-3,0.90,...,0,0,0,0,1,0,0,1,3,10
